# Introduction

This notebook contains my test answers to SQL problems from TestDome and Interview Query.

In [2]:
import sqlite3
import pandas as pd

## TestDome - Regional Sales Comparison

An insurance company maintains records of sales made by its employees. Each employee is assigned to a state. States are grouped under regions.

Write a query that returns:

1. The region name.
2. Average sales per employee for the region (Average sales = Total sales made for the region / Number of employees in the region).
3. The difference between the average sales of the region with the highest average sales, and the average sales per employee for the region (average sales to be calculated as explained above).

Employees can have multiple sales. A region with no sales should be also returned. Use 0 for average sales per employee for such a region when calculating the 2nd and the 3rd column.

In [3]:
conn = sqlite3.connect('dbs/regional_sales.db')
cur = conn.cursor()

In [11]:
# example data

setup_qs = ["""
CREATE TABLE IF NOT EXISTS regions( id INTEGER PRIMARY KEY, name VARCHAR(50) NOT NULL );
""","""
CREATE TABLE IF NOT EXISTS states( id INTEGER PRIMARY KEY, name VARCHAR(50) NOT NULL, regionId INTEGER NOT NULL REFERENCES regions(id) );
""","""
CREATE TABLE IF NOT EXISTS employees ( id INTEGER PRIMARY KEY, name VARCHAR(50) NOT NULL, stateId INTEGER NOT NULL REFERENCES states(id) );
""","""
CREATE TABLE IF NOT EXISTS sales ( id INTEGER PRIMARY KEY, amount INTEGER NOT NULL, employeeId INTEGER NOT NULL REFERENCES employees(id) );
""","""
INSERT INTO regions(id, name) VALUES(1, 'North'); ""","""
INSERT INTO regions(id, name) VALUES(2, 'South'); ""","""
INSERT INTO regions(id, name) VALUES(3, 'East'); ""","""
INSERT INTO regions(id, name) VALUES(4, 'West'); ""","""
INSERT INTO regions(id, name) VALUES(5, 'Midwest');
""","""
INSERT INTO states(id, name, regionId) VALUES(1, 'Minnesota', 1); ""","""
INSERT INTO states(id, name, regionId) VALUES(2, 'Texas', 2); ""","""
INSERT INTO states(id, name, regionId) VALUES(3, 'California', 3); ""","""
INSERT INTO states(id, name, regionId) VALUES(4, 'Columbia', 4); ""","""
INSERT INTO states(id, name, regionId) VALUES(5, 'Indiana', 5);
""","""
INSERT INTO employees(id, name, stateId) VALUES(1, 'Jaden', 1); ""","""
INSERT INTO employees(id, name, stateId) VALUES(2, 'Abby', 1); ""","""
INSERT INTO employees(id, name, stateId) VALUES(3, 'Amaya', 2); ""","""
INSERT INTO employees(id, name, stateId) VALUES(4, 'Robert', 3); ""","""
INSERT INTO employees(id, name, stateId) VALUES(5, 'Tom', 4); ""","""
INSERT INTO employees(id, name, stateId) VALUES(6, 'William', 5);
""","""
INSERT INTO sales(id, amount, employeeId) VALUES(1, 2000, 1); ""","""
INSERT INTO sales(id, amount, employeeId) VALUES(2, 3000, 2); ""","""
INSERT INTO sales(id, amount, employeeId) VALUES(3, 4000, 3); ""","""
INSERT INTO sales(id, amount, employeeId) VALUES(4, 1200, 4); ""","""
INSERT INTO sales(id, amount, employeeId) VALUES(5, 2400, 5);
"""]

[cur.execute(q) for q in setup_qs]

In [22]:
# my answer

cur.execute("""
WITH 
    region_sales (region, avg_sales_per_employee)
    AS
    (
        SELECT r.name
            , COALESCE(SUM(sa.amount) / COUNT(DISTINCT e.id), 0)
        FROM regions r
        LEFT JOIN states st ON st.regionId = r.id
        LEFT JOIN employees e ON e.stateId = st.id
        LEFT JOIN sales sa ON sa.employeeId = e.id
        GROUP BY r.id
    )
SELECT region
    , COALESCE(avg_sales_per_employee, 0) AS regional_avg
    , COALESCE((SELECT MAX(avg_sales_per_employee) FROM region_sales) - avg_sales_per_employee, 0) as diff_from_max
FROM region_sales
""")

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,region,regional_avg,diff_from_max
0,North,2500,1500
1,South,4000,0
2,East,1200,2800
3,West,2400,1600
4,Midwest,0,4000


## IQ - Search Ratings

`search_results` table

|column|type|
|--|--|
|query|varchar|
|result_id|integer|
|position|integer|
|rating|integer|
 

You're given a table that represents search results from searches on Facebook. The query column is the search term, position column represents each position the search result came in, and the rating column represents the human rating of the search result from 1 to 5 where 5 is high relevance and 1 is low relevance.

1. Write a query to compute a metric to measure the quality of the search results for each query. 

2. You want to be able to compute a metric that measures the precision of the ranking system based on position. For example, if the results for dog and cat are....

query|result_id|position|rating|notes
--|--|--|--|--
dog|1000|1|2|picture of hotdog
dog|998|2|4|dog walking
dog|342|3|1|zebra
cat|123|1|4|picture of cat
cat|435|2|2|cat memes
cat|545|3|1|pizza shops
 

...we would rank 'cat' as having a better search result ranking precision than 'dog' based on the correct sorting by rating.

Write a query to create a metric that can validate and rank the queries by their search result precision. Round the metric (avg_rating column) to 2 decimal places.

Output:

column|type
--|--
query|varchar
avg_rating|float

In [23]:
conn = sqlite3.connect('dbs/search_ratings.db')
cur = conn.cursor()

In [24]:
setup_qs = ["""
CREATE TABLE IF NOT EXISTS search_results(
result_id INTEGER NOT NULL, 
query VARCHAR(50) NOT NULL,
position INTEGER NOT NULL,
rating INTEGER);
""","""
INSERT INTO search_results(query, result_id, position, rating) VALUES('dog', 1000, 1, 2); ""","""
INSERT INTO search_results(query, result_id, position, rating) VALUES('dog', 998, 2, 4); ""","""
INSERT INTO search_results(query, result_id, position, rating) VALUES('dog', 342, 3, 1); ""","""
INSERT INTO search_results(query, result_id, position, rating) VALUES('cat', 123, 1, 4); ""","""
INSERT INTO search_results(query, result_id, position, rating) VALUES('cat', 435, 2, 2); ""","""
INSERT INTO search_results(query, result_id, position, rating) VALUES('cat', 545, 3, 1); 
"""]

[cur.execute(q) for q in setup_qs]

A set of 5 search results with the ideal positions would be:

position|rating
--|--
1|5
2|4
3|3
4|2
5|1

If we divided rating by position, the perfect score would be 5+2+1+0.5+0.2 = 8.7. Divided by count it would be 1.74

The worst score would be 5. Divided by count it would be 1.

position|rating
--|--
1|1
2|2
3|3
4|4
5|5

The dog score in the example with 3 results would be 2+2+0.33 = 4.33. Divided by count it would be 1.44.

The cat score would be 4+1+0.33 = 5.33. Divided by count it would be 1.77

But the scores will be different for different numbers of search results. What if we divided by the count?

I like the divide by count better, since it normalizes the scores for different numbers of results.

In [39]:
cur.execute("""
SELECT query
    , ROUND(SUM(CAST(rating AS FLOAT) / position) / COUNT(rating), 2) AS avg_rating
FROM search_results
GROUP BY query
ORDER BY avg_rating DESC
""")

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,query,avg_rating
0,cat,1.78
1,dog,1.44


## IQ - Paired Products

`transactions` table

column|type
--|--
id|integer
user_id|integer
created_at|datetime
product_id|integer
quantity|integer

`products` table

column|type
--|--
id|integer
name|string
price|float
 

Let's say we have two tables, `transactions` and `products`. Hypothetically the `transactions` table consists of over a billion rows of purchases bought by users.

We are trying to find paired products that are often purchased together by the same user, such as wine and bottle openers, chips and beer, etc..

Write a query to find the top five paired products and their names.

Note: for the purposes of satisfying the test case, P1 should be the item that comes first in the alphabet.

Output:

column|type
--|--
P1|string
P2|string
count|integer

In [ ]:
conn = sqlite3.connect('dbs/paired_products.db')
cur = conn.cursor()

setup_qs = ["""
CREATE TABLE IF NOT EXISTS transactions(
id
result_id INTEGER NOT NULL, 
query VARCHAR(50) NOT NULL,
position INTEGER NOT NULL,
rating INTEGER);
""","""
INSERT INTO search_results(query, result_id, position, rating) VALUES('dog', 1000, 1, 2); ""","""
INSERT INTO search_results(querby, result_id, position, rating) VALUES('dog', 998, 2, 4); ""","""
INSERT INTO search_results(query, result_id, position, rating) VALUES('dog', 342, 3, 1); ""","""
INSERT INTO search_results(query, result_id, position, rating) VALUES('cat', 123, 1, 4); ""","""
INSERT INTO search_results(query, result_id, position, rating) VALUES('cat', 435, 2, 2); ""","""
INSERT INTO search_results(query, result_id, position, rating) VALUES('cat', 545, 3, 1); 
"""]

[cur.execute(q) for q in setup_qs]

# Practicing with Group By

Example data from https://www.mssqltips.com/sqlservertip/6315/group-by-in-sql-server-with-cube-rollup-and-grouping-sets-examples/

In [3]:
conn = sqlite3.connect('dbs/salaries.db')
cur = conn.cursor()

In [17]:
cur.execute("""DELETE FROM EmpSalary""")

In [18]:
setup_qs = ["""
CREATE TABLE IF NOT EXISTS EmpSalary(
  id INTEGER PRIMARY KEY, 
  EmpName varchar (200), 
  Department varchar(100), 
  Category char(1), 
  Salary INTEGER );
""","""
INSERT INTO EmpSalary(EmpName, Department, Category, Salary) VALUES('Bhavesh Patel','IT','A',8000); ""","""
INSERT INTO EmpSalary(EmpName, Department, Category, Salary) VALUES('Alpesh Patel','Sales','A',7000); ""","""
INSERT INTO EmpSalary(EmpName, Department, Category, Salary) VALUES('Kalpesh Thakor','IT','B',5000 ); ""","""
INSERT INTO EmpSalary(EmpName, Department, Category, Salary) VALUES('Jay Shah','Sales','B',4000 ); ""","""
INSERT INTO EmpSalary(EmpName, Department, Category, Salary) VALUES('Ram Nayak','IT','C',3000); ""","""
INSERT INTO EmpSalary(EmpName, Department, Category, Salary) VALUES('Jay Shaw','Sales','C',2000); ""","""
INSERT INTO EmpSalary(EmpName, Department, Category, Salary) VALUES('John Snow','IT','B',6000); ""","""
INSERT INTO EmpSalary(EmpName, Department, Category, Salary) VALUES('Arya Stark','IT','A',5500); ""","""
INSERT INTO EmpSalary(EmpName, Department, Category, Salary) VALUES('Homer Simpson','Sales','C',3000); ""","""
INSERT INTO EmpSalary(EmpName, Department, Category, Salary) VALUES('Geoffrey Baratheon','Shipping','D',4500); ""","""
INSERT INTO EmpSalary(EmpName, Department, Category, Salary) VALUES('Lisa Simpson','IT','C',2000); """]

[cur.execute(q) for q in setup_qs]


In [22]:
cur.execute("""
SELECT Department, Category, count() AS Employee_Count
FROM EmpSalary
GROUP BY Department, Category
""")

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,Department,Category,Employee_Count
0,IT,A,2
1,IT,B,2
2,IT,C,2
3,Sales,A,1
4,Sales,B,1
5,Sales,C,2
6,Shipping,D,1


What if I wanted the count of employees whose salaries were above the average for the department, grouped by department?

In [31]:
cur.execute("""
SELECT e.Department, e.Category, count() AS Employee_Count,
    ROUND(a.Avg_Salary, 2) Avg_Salary_Per_Dept
FROM EmpSalary e,
    (SELECT avg(Salary) Avg_Salary, Department 
     FROM EmpSalary 
     GROUP BY Department) a
WHERE e.Department = a.Department 
AND e.Salary > a.Avg_Salary 
GROUP BY e.Department
""")

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,Department,Category,Employee_Count,Avg_Salary_Per_Dept
0,IT,A,4,4916.67
1,Sales,A,1,4000.00
